# Setup of Main


In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt



# Data generation for the copy task.
def generate_copy_data(num_samples, T, vocab_size):
    """
    For each sample:
      - Generate a random sequence of length T (tokens from 1 to vocab_size).
      - Build input: [sequence] + [delimiter token] + [T blank tokens (0)]
      - Build target: [T+1 blanks] + [the original sequence]
    """
    seq_length = 2 * T + 1
    X = np.zeros((num_samples, seq_length), dtype=np.int32)
    Y = np.zeros((num_samples, seq_length), dtype=np.int32)
    delimiter_token = vocab_size + 1  # special delimiter token
    for i in range(num_samples):
        # Random sequence of T tokens (range: 1 to vocab_size)
        random_seq = np.random.randint(1, vocab_size + 1, size=T)
        # Input: first T tokens are the sequence, then a delimiter, then T blanks (0)
        X[i, :T] = random_seq
        X[i, T] = delimiter_token
        X[i, T+1:] = 0
        # Target: first T+1 positions are blanks (0), last T positions are the sequence
        Y[i, :T+1] = 0
        Y[i, T+1:] = random_seq
    return X, Y

# Hyperparameters
num_samples = 10000    # total examples
T = 10                # sequence length to copy (experiment with larger values as needed)
vocab_size = 10       # tokens 1..10 (e.g., representing letters a-j)
hidden_size = 128
batch_size = 128
epochs = 20

delimiter_token = vocab_size + 1  # used for both data generation and model processing
blank_token = 0

# Generate the copy task data.
X, Y = generate_copy_data(num_samples, T, vocab_size)

# Split into training (70%) and validation (30%).
split_index = int(0.7 * num_samples)
X_train, Y_train = X[:split_index], Y[:split_index]
X_val, Y_val = X[split_index:], Y[split_index:]

# Create tf.data.Dataset objects.
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(1000).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(batch_size)



# STD LSTM (tensorflow)

In [7]:
class STDLSTMCell(tf.keras.layers.Layer):
    def __init__(self, units):
        super(STDLSTMCell, self).__init__()
        self.units = units
        # Define the state size: hidden state and cell state.
        self.state_size = [self.units, self.units]
        self.output_size = self.units

    def build(self, input_shape):
        input_dim = input_shape[-1]
        # Input gate weights and bias
        self.W_i = self.add_weight(shape=(input_dim, self.units), initializer='random_normal', trainable=True)
        self.U_i = self.add_weight(shape=(self.units, self.units), initializer='random_normal', trainable=True)
        self.b_i = self.add_weight(shape=(self.units,), initializer='zeros', trainable=True)
        # Forget gate weights and bias
        self.W_f = self.add_weight(shape=(input_dim, self.units), initializer='random_normal', trainable=True)
        self.U_f = self.add_weight(shape=(self.units, self.units), initializer='random_normal', trainable=True)
        self.b_f = self.add_weight(shape=(self.units,), initializer='zeros', trainable=True)
        # Output gate weights and bias
        self.W_o = self.add_weight(shape=(input_dim, self.units), initializer='random_normal', trainable=True)
        self.U_o = self.add_weight(shape=(self.units, self.units), initializer='random_normal', trainable=True)
        self.b_o = self.add_weight(shape=(self.units,), initializer='zeros', trainable=True)
        # Candidate cell state weights and bias
        self.W_c = self.add_weight(shape=(input_dim, self.units), initializer='random_normal', trainable=True)
        self.U_c = self.add_weight(shape=(self.units, self.units), initializer='random_normal', trainable=True)
        self.b_c = self.add_weight(shape=(self.units,), initializer='zeros', trainable=True)

    def call(self, x, states):
        h_prev, c_prev = states

        # Input gate: i_t = sigmoid(W_i * x_t + U_i * h_(t-1) + b_i)
        i = tf.sigmoid(tf.matmul(x, self.W_i) + tf.matmul(h_prev, self.U_i) + self.b_i)
        # Forget gate: f_t = sigmoid(W_f * x_t + U_f * h_(t-1) + b_f)
        f = tf.sigmoid(tf.matmul(x, self.W_f) + tf.matmul(h_prev, self.U_f) + self.b_f)
        # Output gate: o_t = sigmoid(W_o * x_t + U_o * h_(t-1) + b_o)
        o = tf.sigmoid(tf.matmul(x, self.W_o) + tf.matmul(h_prev, self.U_o) + self.b_o)
        # Candidate cell state: c̃_t = tanh(W_c * x_t + U_c * h_(t-1) + b_c)
        c_hat = tf.tanh(tf.matmul(x, self.W_c) + tf.matmul(h_prev, self.U_c) + self.b_c)
        # New cell state: c_t = f_t ⊙ c_(t-1) + i_t ⊙ c̃_t
        c = f * c_prev + i * c_hat
        # New hidden state: h_t = o_t ⊙ tanh(c_t)
        h = o * tf.tanh(c)

        return h, [h, c]

# Custom LSTM model that uses the above custom cell.
class STDLSTMModel(tf.keras.Model):
    def __init__(self, vocab_size, hidden_size):
        super(STDLSTMModel, self).__init__()
        # The embedding layer maps input tokens to vectors.
        # We use vocab_size+2 because token 0 is blank and token (vocab_size+1) is the delimiter.
        self.embedding = tf.keras.layers.Embedding(vocab_size + 2, hidden_size)
        # Wrap the custom LSTM cell with tf.keras.layers.RNN.
        self.rnn = tf.keras.layers.RNN(STDLSTMCell(hidden_size), return_sequences=True, return_state=True)
        # Dense layer to project RNN outputs to (vocab_size+1) probabilities.
        self.dense = tf.keras.layers.Dense(vocab_size + 1, activation='softmax')

    def call(self, inputs, initial_state=None, training=False):
        x = self.embedding(inputs)
        if initial_state is None:
            rnn_out, h, c = self.rnn(x)
        else:
            rnn_out, h, c = self.rnn(x, initial_state=initial_state)
        output = self.dense(rnn_out)
        return output

Train the Model

In [ ]:
# Instantiate and compile the custom LSTM model.
custom_model = STDLSTMModel(vocab_size, hidden_size)
custom_model.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

# Train the model.
history = custom_model.fit(train_dataset, epochs=epochs, validation_data=val_dataset)

# Plot training loss and accuracy.
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], marker='o', label='Train Loss')
plt.plot(history.history['val_loss'], marker='o', label='Validation Loss')
plt.title('Loss vs. Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], marker='o', label='Train Accuracy')
plt.plot(history.history['val_accuracy'], marker='o', label='Validation Accuracy')
plt.title('Accuracy vs. Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


# MUL LSTM (tensorflow)

In [ ]:
class MLSTMCell(tf.keras.layers.Layer):
    def __init__(self, units):
        super(MLSTMCell, self).__init__()
        self.units = units
        # State size: hidden state (h) and cell state (c).
        self.state_size = [self.units, self.units]
        self.output_size = self.units

    def build(self, input_shape):
        input_dim = input_shape[-1]
        # -- Multiplicative term parameters: m_t = W_m x_t + U_m h_(t-1) + b_m --
        # Produces a vector of size [batch_size, input_dim], which is then
        # elementwise-multiplied with x_t to form x̃_t.
        self.W_m = self.add_weight( shape=(input_dim, input_dim), initializer="random_normal", trainable=True, name="W_m",)
        self.U_m = self.add_weight( shape=(self.units, input_dim), initializer="random_normal", trainable=True, name="U_m",)
        self.b_m = self.add_weight( shape=(input_dim,), initializer="zeros", trainable=True, name="b_m")

        # -- LSTM gate parameters (same shapes as standard LSTM) --
        # Input gate
        self.W_i = self.add_weight( shape=(input_dim, self.units), initializer="random_normal", trainable=True, name="W_i",)
        self.U_i = self.add_weight( shape=(self.units, self.units), initializer="random_normal", trainable=True, name="U_i",)
        self.b_i = self.add_weight( shape=(self.units,), initializer="zeros", trainable=True, name="b_i")

        # Forget gate
        self.W_f = self.add_weight( shape=(input_dim, self.units), initializer="random_normal", trainable=True, name="W_f",)
        self.U_f = self.add_weight( shape=(self.units, self.units), initializer="random_normal", trainable=True, name="U_f",)
        self.b_f = self.add_weight( shape=(self.units,), initializer="zeros", trainable=True, name="b_f")

        # Output gate
        self.W_o = self.add_weight( shape=(input_dim, self.units), initializer="random_normal", trainable=True, name="W_o",)
        self.U_o = self.add_weight( shape=(self.units, self.units), initializer="random_normal", trainable=True, name="U_o",)
        self.b_o = self.add_weight( shape=(self.units,), initializer="zeros", trainable=True, name="b_o")

        # Candidate cell state
        self.W_c = self.add_weight( shape=(input_dim, self.units), initializer="random_normal", trainable=True, name="W_c",)
        self.U_c = self.add_weight( shape=(self.units, self.units), initializer="random_normal", trainable=True, name="U_c",)
        self.b_c = self.add_weight( shape=(self.units,), initializer="zeros", trainable=True, name="b_c")

    def call(self, x, states):
        """
        x:      [batch_size, input_dim]
        states: [h_prev, c_prev], each of shape [batch_size, units]
        """
        h_prev, c_prev = states
        #  m_t = W_m * x + U_m * h_prev + b_m
        m_t = tf.matmul(x, self.W_m) + tf.matmul(h_prev, self.U_m) + self.b_m
        #  x̃_t = m_t ⊙ x
        x_tilde = m_t * x  # shape: [batch_size, input_dim]
        # Input gate: i_t
        i = tf.sigmoid(tf.matmul(x_tilde, self.W_i) + tf.matmul(h_prev, self.U_i) + self.b_i)
        # Forget gate: f_t
        f = tf.sigmoid(tf.matmul(x_tilde, self.W_f) + tf.matmul(h_prev, self.U_f) + self.b_f)
        # Output gate: o_t
        o = tf.sigmoid(tf.matmul(x_tilde, self.W_o) + tf.matmul(h_prev, self.U_o) + self.b_o)
        # Candidate cell state: c̃_t
        c_hat = tf.tanh(tf.matmul(x_tilde, self.W_c) + tf.matmul(h_prev, self.U_c) + self.b_c)
        # New cell state: c_t
        c = f * c_prev + i * c_hat
        # New hidden state: h_t
        h = o * tf.tanh(c)
        return h, [h, c]

class MLSTMModel(tf.keras.Model):
    def __init__(self, vocab_size, hidden_size):
        super(MLSTMModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size + 2, hidden_size)
        self.rnn = tf.keras.layers.RNN(
            MLSTMCell(hidden_size),
            return_sequences=True,
            return_state=True
        )
        # Dense layer to project RNN outputs to (vocab_size+1) probabilities.
        self.dense = tf.keras.layers.Dense(vocab_size + 1, activation='softmax')

    def call(self, inputs, initial_state=None, training=False):
        x = self.embedding(inputs)
        if initial_state is None:
            rnn_out, h, c = self.rnn(x)  # automatically uses zeros if no init state
        else:
            rnn_out, h, c = self.rnn(x, initial_state=initial_state)
        output = self.dense(rnn_out)
        return output

# STD GRU (tensorflow)

In [ ]:
class StandardGRUCell(tf.keras.layers.Layer):
    """
    A custom implementation of the standard GRU cell using TensorFlow,
    following the equations:

        z_t = sigma(W_z * x_t + U_z * h_{t-1} + b_z)
        r_t = sigma(W_r * x_t + U_r * h_{t-1} + b_r)
        h_tilde = tanh(W_h * x_t + U_h * (r_t * h_{t-1}) + b_h)
        h_t = (1 - z_t) * h_{t-1} + z_t * h_tilde
    """
    def __init__(self, units, **kwargs):
        super(StandardGRUCell, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        # input_shape is (batch_size, input_dim)
        input_dim = input_shape[-1]
        self.Wz = self.add_weight( shape=(input_dim, self.units), initializer='glorot_uniform', name='Wz' )
        self.Uz = self.add_weight( shape=(self.units, self.units), initializer='orthogonal', name='Uz' )
        self.bz = self.add_weight( shape=(self.units,), initializer='zeros', name='bz' )
        # Weights for reset gate r
        self.Wr = self.add_weight( shape=(input_dim, self.units), initializer='glorot_uniform', name='Wr' )
        self.Ur = self.add_weight( shape=(self.units, self.units), initializer='orthogonal', name='Ur' )
        self.br = self.add_weight( shape=(self.units,), initializer='zeros', name='br' )
        # Weights for candidate hidden state h_tilde
        self.Wh = self.add_weight( shape=(input_dim, self.units), initializer='glorot_uniform', name='Wh' )
        self.Uh = self.add_weight( shape=(self.units, self.units), initializer='orthogonal', name='Uh' )
        self.bh = self.add_weight( shape=(self.units,), initializer='zeros', name='bh' )
        super(StandardGRUCell, self).build(input_shape)

    def call(self, inputs, states):
        """
        inputs: (batch_size, input_dim)
        states: list with a single element h_{t-1} of shape (batch_size, units)
        """
        h_prev = states[0]
        z_t = tf.sigmoid( tf.matmul(inputs, self.Wz) + tf.matmul(h_prev, self.Uz) + self.bz )
        # Reset gate r_t
        r_t = tf.sigmoid( tf.matmul(inputs, self.Wr) + tf.matmul(h_prev, self.Ur) + self.br )
        # Candidate hidden state h_tilde
        h_tilde = tf.tanh( tf.matmul(inputs, self.Wh) + tf.matmul(r_t * h_prev, self.Uh) + self.bh )
        # Final new hidden state h_t
        h_t = (1 - z_t) * h_prev + z_t * h_tilde
        return h_t, [h_t]

# MUL GRU (tensorflow)

In [ ]:
class MultiplicativeGRUCell(tf.keras.layers.Layer):
    """
    A custom implementation of the multiplicative GRU cell using TensorFlow.

    Equations:
        m_t = W_m * x_t + U_m * h_{t-1} + b_m
        x_tilde = m_t ⊙ x_t

        z_t = sigma(W_z * x_tilde + U_z * h_{t-1} + b_z)
        r_t = sigma(W_r * x_tilde + U_r * h_{t-1} + b_r)
        h_tilde = tanh(W_h * x_tilde + U_h * (r_t ⊙ h_{t-1}) + b_h)
        h_t = (1 - z_t) ⊙ h_{t-1} + z_t ⊙ h_tilde
    """

    def __init__(self, units, **kwargs):
        super(MultiplicativeGRUCell, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        """
        input_shape is typically (batch_size, input_dim)
        """
        input_dim = input_shape[-1]

        # Weights for m_t
        self.Wm = self.add_weight( shape=(input_dim, self.units), initializer='glorot_uniform', name='Wm' )
        self.Um = self.add_weight( shape=(self.units, self.units), initializer='orthogonal', name='Um' )
        self.bm = self.add_weight( shape=(self.units,), initializer='zeros', name='bm' )

        # Weights for update gate z
        self.Wz = self.add_weight( shape=(self.units, self.units), initializer='glorot_uniform', name='Wz' )
        self.Uz = self.add_weight( shape=(self.units, self.units), initializer='orthogonal', name='Uz' )
        self.bz = self.add_weight( shape=(self.units,), initializer='zeros', name='bz' )

        # Weights for reset gate r
        self.Wr = self.add_weight( shape=(self.units, self.units), initializer='glorot_uniform', name='Wr' )
        self.Ur = self.add_weight( shape=(self.units, self.units), initializer='orthogonal', name='Ur' )
        self.br = self.add_weight( shape=(self.units,), initializer='zeros', name='br' )

        # Weights for candidate hidden state
        self.Wh = self.add_weight( shape=(self.units, self.units), initializer='glorot_uniform', name='Wh' )
        self.Uh = self.add_weight( shape=(self.units, self.units), initializer='orthogonal', name='Uh' )
        self.bh = self.add_weight( shape=(self.units,), initializer='zeros', name='bh' )

        self.input_dim = input_dim

        super(MultiplicativeGRUCell, self).build(input_shape)

    def call(self, inputs, states):
        """
        inputs: (batch_size, input_dim)
        states: list with a single element h_{t-1} of shape (batch_size, units)
        """
        h_prev = states[0]  # shape: (batch_size, units)

        # 1) Compute m_t of shape (batch_size, units)
        #    m_t = W_m x_t + U_m h_{t-1} + b_m
        m_t = (
            tf.matmul(inputs, self.Wm) +
            tf.matmul(h_prev, self.Um) +
            self.bm
        )  # shape: (batch_size, units)

        # 2) x_tilde = m_t ⊙ x_t
        #    But we have a shape mismatch: m_t is (batch_size, units),
        #    while x_t is (batch_size, input_dim).
        #
        #    Typically, "multiplicative" gating is done elementwise,
        #    so we need them to be the same dimension.
        #    If input_dim == units, then we can do elementwise multiplication.
        #
        #    If they differ, you might do a diagonal approach or another transform.
        #
        #    For simplicity, let's assume input_dim == units for demonstration.
        #    If your input_dim != units, you need a consistent design (like a diagonal of size input_dim).
        #
        #    Let's assume input_dim == units for this example.
        x_tilde = m_t * inputs  # elementwise multiplication
        # 3) Update gate z_t
        z_t = tf.sigmoid( tf.matmul(x_tilde, self.Wz) + tf.matmul(h_prev, self.Uz) + self.bz )
        # 4) Reset gate r_t
        r_t = tf.sigmoid( tf.matmul(x_tilde, self.Wr) + tf.matmul(h_prev, self.Ur) + self.br )
        # 5) Candidate hidden state h_tilde
        h_tilde = tf.tanh( tf.matmul(x_tilde, self.Wh) + tf.matmul(r_t * h_prev, self.Uh) + self.bh )
        # 6) Final new hidden state h_t
        h_t = (1 - z_t) * h_prev + z_t * h_tilde
        return h_t, [h_t]